Test if AGN metrics can return a worse value even if they are given more data

In [1]:
import numpy as np
import rubin_sim.maf.metrics as metrics

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

import rubin_sim.maf as maf
import rubin_scheduler.utils as rsUtils
from rubin_sim.data import get_baseline


In [2]:
baseline_file = get_baseline()
name = os.path.basename(baseline_file).replace('.db','')
out_dir = 'temp'
results_db = maf.db.ResultsDb(out_dir=out_dir)

bundle_list = []
# The point on the sky we would like to get visits for
ra = [0.]
dec = [-20]


# Say we just want to pass data through, not compute anything. Documentation on 
# columns at:  https://rubin-sim.lsst.io/rs_scheduler/output_schema.html
metric = maf.metrics.PassMetric(cols=['filter', 'observationStartMJD', 'fiveSigmaDepth', 'visitExposureTime'])
# Select all the visits. Could be something like "filter='r'", "night < 365", etc
sql = ''
slicer = maf.slicers.UserPointsSlicer(ra=ra, dec=dec)
bundle_list.append(maf.MetricBundle(metric, slicer, sql, run_name=name))

bd = maf.metricBundles.make_bundles_dict_from_list(bundle_list)
bg = maf.metricBundles.MetricBundleGroup(bd, baseline_file, out_dir=out_dir, results_db=results_db)
bg.run_all()


data_slice = bundle_list[0].metric_values[0]


In [3]:
agn_m5 = {"u": 22.89, "g": 23.94, "r": 23.5, "i": 22.93, "z": 22.28, "y": 21.5}
f = "g"
m = metrics.SFUncertMetric(
            mag=agn_m5[f],
            bins=np.logspace(0, np.log10(3650), 16),
            metric_name="AGN SF_uncert",
        )

In [4]:
# run the metric with all the data
in_g = np.where(data_slice["filter"] == "g")
sp = {}
sp["ebv"] = 0.
result_all = m.run(data_slice[in_g], slice_point=sp)
result_all

np.float64(0.03112752290577717)

In [5]:
# run excluding the fist point
sp = {}
sp["ebv"] = 0.
result_m1 = m.run(data_slice[in_g][1:], slice_point=sp)
result_m1

np.float64(0.031097001346329995)

In [6]:
# This is computing an uncertainty, so adding data should make it go down
assert result_all < result_m1

AssertionError: 

In [7]:
# and we have an error, this metric is bugged

In [8]:
nquist_threshold = 2.2
lag = 100
m = metrics.AgnTimeLagMetric(threshold=nquist_threshold, lag=lag)

In [9]:
m.run(data_slice, slice_point=sp)

np.float64(10.717038813302244)

In [10]:
# make a larger data_set
data_2 = data_slice.copy()
data_2["observationStartMJD"] = data_2["observationStartMJD"].max() + np.arange(0,data_2.size)*5

data_large = np.concatenate([data_slice, data_2])


In [11]:
m.run(data_large, slice_point=sp)

np.float64(10.35274278975786)